In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import time
import random

In [ ]:
# t_data => BGR input, 1st dense layer o/p
#t_data2 => RGB input, 1st dense layer o/p
#t_data3 => RGB i/p, 2nd dense layer o/p but extension on mean image
#t_data4 => RGB i/p, 2nd dense layer o/p (extension on total pics) - saved time on test folder by min(pics per biz_id , 100)

In [ ]:
train = pd.read_csv("F:\sharath\yelpdata\\train_data_4.csv")
test = pd.read_csv("F:\sharath\yelpdata\\test_data_4.csv")

In [ ]:
def get_train_test_data(file1, file2):
    
    path1 = "F:\sharath\yelpdata\\"+file1
    path2 = "F:\sharath\yelpdata\\"+file2
    train = pd.read_csv(path1)
    test = pd.read_csv(path2)

    #Create train data for classification
    X = train.ix[:,"Features"]
    X = map((lambda x: [float(f) for f in x[1:-1].split(",")]), X)
    X = np.array(X)

    #binarize the labels for one Vs rest classifier fit
    y = train.ix[:,"Target"]
    y = map((lambda x: [int(i) for i in x[1:-1].split(",")]), y)
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(y)

    #Create test data for prediction
    X_test = test.ix[:,"Features"]
    X_test = map((lambda x: [float(f) for f in x[1:-1].split(",")]), X_test)
    X_test = np.array(X_test)
    
    print X.shape, y.shape
    print X_test.shape

    return X,y,X_test,mlb,test

In [ ]:
def cross_validate(X,y):    
    f1_score_svm = []
    f1_score_rf = []
    kf = KFold(len(X), 3, shuffle=True)
    for tr_idx, val_idx in kf:
        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        print X_tr.shape, y_tr.shape
        print X_val.shape, y_val.shape

        #Fit SVM - OneVsRest
        start = time.time()
        clf = OneVsRestClassifier(svm.SVC(kernel="linear", probability=True))
        clf.fit(X_tr, y_tr)
        #print("time for fitting svm model %d" %(time.time()-start))

        start = time.time()
        pred = clf.predict(X_val)
        #print("time for prediction %d" %(time.time()-start))
        
        #Calculate average f1 score
        f1 = round(f1_score(y_val, pred, average="micro"),3)
        print("f1 score of svm %0.3f" %(f1))
        f1_score_svm.append(f1)

        #Fit Random Forest - OneVsRest
        start = time.time()
        rf = RandomForestClassifier(n_estimators=400, max_depth=150, verbose=0)
        rf.fit(X_tr, y_tr)
        #print("time for fitting rf model %d" %(time.time()-start))

        start = time.time()
        pred = rf.predict(X_val)
        #print("time for prediction %d" %(time.time()-start))
        
        #Calculate average f1 score
        f1 = round(f1_score(y_val, pred, average="micro"),3)
        print ("f1 score of rf %0.3f" %(f1))
        f1_score_rf.append(f1)

    print ("svm scores:", f1_score_svm)
    print ("rf scores:", f1_score_rf)
    return f1_score_svm,f1_score_rf

In [ ]:
# setup a cv - data2
X,y,X_test2,mlb,test = get_train_test_data("train_data_2.csv", "test_data_2.csv")
print("CV scores on t_data2:\n")
f1_sv, f1_rf = cross_validate(X,y)

with open("results.txt", "a") as f:
    f.write("\nt_data2 results: \n")
    f.write("svm scores: " + str(f1_sv))
    f.write("\nrf scores: " + str(f1_rf))
f.close()

# setup a cv - data3
X,y,X_test3,mlb,test = get_train_test_data("train_data_3.csv", "test_data_3.csv")
print("CV scores on t_data3:\n")
f1_sv, f1_rf = cross_validate(X,y)
with open("results.txt", "a") as f:
    f.write("\nt_data3 results: \n")
    f.write("svm scores: " + str(f1_sv))
    f.write("\nrf scores: " + str(f1_rf))
f.close()

# setup a cv - data4
X,y,X_test4,mlb,test = get_train_test_data("train_data_4.csv", "test_data_4.csv")
print("CV scores on t_data4:\n")
f1_sv, f1_rf = cross_validate(X,y)
with open("results.txt", "a") as f:
    f.write("\nt_data4 results: \n")
    f.write("svm scores: " + str(f1_sv))
    f.write("\nrf scores: " + str(f1_rf))
f.close()

# setup a cv - data1
X,y,X_test,mlb,test = get_train_test_data("train_data.csv", "test_data.csv")
print("CV scores on t_data:\n")
f1_sv, f1_rf = cross_validate(X,y)
with open("results.txt", "a") as f:
    f.write("\nt_data results: \n")
    f.write("svm scores: " + str(f1_sv))
    f.write("\nrf scores: " + str(f1_rf))
f.close()

In [228]:
#training on full train data - Random forest
X,y,X_test,mlb,test = get_train_test_data("train_data_4.csv", "test_data_4.csv")

clf = OneVsRestClassifier(svm.SVC(kernel="linear", probability=True))
clf.fit(X,y)
prob=clf.predict_proba(X_test)
#pred=clf.predict(X_test)

rf = RandomForestClassifier(n_estimators=300, max_depth=500, verbose=1, oob_score=True)
rf.fit(X,y)
#pred_rf = rf.predict(X_test)
prob_rf=rf.predict_proba(X_test)
#Calculate average f1 score
#f1_score(yts, pred, average="micro")

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    7.7s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   32.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   48.5s finished
[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:    3.7s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    5.6s finished
[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:    3.6s


(1996L, 4096L) (1996L, 9L)
(10000L, 4096L)


[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    5.7s finished


In [229]:
prob_rf = np.array(prob_rf)
prob_rf=prob_rf[:,:,1].T
prob_avg = (prob + prob_rf)/2
p_rf = prob_avg >0.42

In [230]:
p_rf.shape

(10000L, 9L)

In [231]:
test_labels = mlb.inverse_transform(p_rf)
test_labels = [list(label) for label in test_labels]
test_labels = map((lambda x: str(x)[1:-1].replace(",", " ")), test_labels)

In [232]:
#Prepare submission file
# test = pd.read_csv("F:\sharath\yelpdata\\test_data_4.csv")
sub = pd.read_csv("sample_submission.csv")
sub1 = sub.copy()
sub1[["business_id"]] = test[["BusinessId"]]
sub1.ix[:,"labels"] = test_labels
sub1 = pd.merge(sub[["business_id"]], sub1, on = "business_id")

In [233]:
sub1.head()

,business_id,labels
0,003sg,1 2 3 5 6 8
1,00er5,2 3 5 6 8
2,00kad,1 2 3 4 5 6 7
3,00mc6,1 2 3 4 5 6 7
4,00q7x,1 2 3 5 6 7 8


In [234]:
sub1.to_csv("sub_avg_svm_rf_4.csv", index = False)

In [ ]:
n=1600
clf = OneVsRestClassifier(svm.SVC(kernel="linear", probability=True))
clf.fit(X[:n,], y[:n,])
prob=clf.predict_proba(X[n:n+300,])
pred=clf.predict(X[n:n+300,])
#f1_score(y[100:200,],prob, average="micro")

In [ ]:
n=1600
rf = RandomForestClassifier(n_estimators=300, max_depth=500, verbose=1)
rf.fit(X[:n,], y[:n,])
prob_rf=rf.predict_proba(X[n:n+300,])
pred_rf=rf.predict(X[n:n+300,])

In [ ]:
print f1_score(y[n:n+300,],pred, average="micro")
print f1_score(y[n:n+300,],pred_rf, average="micro")

In [ ]:
prob_rf = np.array(prob_rf)
p_rf=prob_rf[:,:,1]
p_rf = p_rf.T
p_rf.shape

p,p_rf = prob>0.481, p_rf>0.41
p=p.astype(np.int)
p_rf= p_rf.astype(np.int)
print f1_score(y[n:n+300,],p, average="micro")
print f1_score(y[n:n+300,],p_rf, average="micro")

In [ ]:
prob_avg = (prob+prob_rf[:,:,1].T)/2
p_avg = prob_avg > 0.42
print f1_score(y[n:n+300,],p_avg, average="micro")

In [ ]:
prob_avg[0,]